In [161]:
import pandas as pd
from datetime import datetime
from datetime import timedelta
import math
import numpy as np

In [162]:
TRANSFORMED_DATA_PATH = "results/trajectories/transformed_coords.csv"

In [163]:
df = pd.read_csv(TRANSFORMED_DATA_PATH, index_col=None, header=0)
df.head()

,video_name,frame_id,xmin,ymin,xmax,ymax,velocity,customer_id,x_floorplan,y_floorplan,new_customer_id,camera,x_floorplan_new,y_floorplan_new,time,customer_id_global
0,20220314_1.03_10_0_blurred,258,653.596516,233.180269,732.845525,330.121294,0.000,2383,-359.594824,-48.170086,1002383,1.03,1226.894568,1430.999265,14:47:33,1002383.0
1,20220314_1.03_10_0_blurred,260,653.706260,233.044283,732.741128,329.715728,0.105,2383,-359.589552,-48.161038,1002383,1.03,1227.066483,1431.227170,14:47:33,1002383.0
2,20220314_1.03_10_0_blurred,261,653.599622,233.117391,731.924925,328.843053,0.185,2383,-359.588172,-48.152324,1002383,1.03,1227.111485,1431.446655,14:47:33,1002383.0
3,20220314_1.03_10_0_blurred,264,653.864126,233.172724,732.331825,329.024946,0.128,2383,-359.587332,-48.155114,1002383,1.03,1227.138867,1431.376397,14:47:34,1002383.0
4,20220314_1.03_10_0_blurred,285,653.823460,232.950344,732.490189,329.088096,0.119,2383,-359.586592,-48.153137,1002383,1.03,1227.163013,1431.426174,14:47:37,1002383.0


In [164]:
def re_id_customers(df):
    for customer in df.customer_id_global.unique():
        if customer == 12161191.0:
            last_customer_row = df.loc[df["customer_id_global"] == customer].sort_values(by='time').iloc[-1]
            print("----------LAST CUSTOMER------------")
            print(last_customer_row)
            
            df_similar_time_customers = get_similar_time_customers(df, last_customer_row["time"], 5, last_customer_row["camera"])
            if df_similar_time_customers.empty:
                print("EMPTY 1")
                continue

            similar_position_customers = get_close_customers(df_similar_time_customers, last_customer_row["x_floorplan_new"], last_customer_row["y_floorplan_new"], 10)
            if similar_position_customers.empty:
                print("EMPTY 2")
                continue

            print("----------SIMILAR POSITION CUSTOMERS------------")
            df.loc[similar_position_customers.index, "customer_reid"] = last_customer_row["customer_id_global"]
            print(df[~df['customer_reid'].isna()])
            for customer in similar_position_customers["new_customer_id"].unique():
                df.loc[df["new_customer_id"] == customer, "customer_reid"] = last_customer_row["customer_id_global"]

            print("----------CUSTOMERS WITH SAME ID------------")
            print(df[~df['customer_reid'].isna()])
            break


def get_similar_time_customers(df, time, interval, camera):
    df_other_cameras = df.loc[df["camera"] != camera]
    begin_time = str((datetime.strptime(time,"%H:%M:%S") - timedelta(seconds=interval)).time())
    end_time = str((datetime.strptime(time,"%H:%M:%S") + timedelta(seconds=interval)).time())
    df_time_interval = get_data_by_time(df_other_cameras, begin_time, end_time)
    return df_time_interval


def get_close_customers(df, xcoord, ycoord, max_dist):
    df_close_customers = df[df.apply(lambda row: math.dist((xcoord, ycoord), (row["x_floorplan_new"], row["y_floorplan_new"])), axis=1) < max_dist]          
    return df_close_customers

    
def get_data_by_time(df, begin_time, end_time):
    return df.loc[(df["time"] > begin_time) & (df["time"] < end_time)]


In [165]:
re_id_customers(df)

----------LAST CUSTOMER------------
video_name            20220314_1.03_12_16_blurred
frame_id                                      395
xmin                                   196.077626
ymin                                   257.286244
xmax                                   304.076501
ymax                                   418.346752
velocity                                    1.053
customer_id                                  1191
x_floorplan                           -363.111867
y_floorplan                            -46.572619
new_customer_id                          12161191
camera                                       1.03
x_floorplan_new                       1112.201002
y_floorplan_new                        1471.23663
time                                     16:30:32
customer_id_global                     12161191.0
Name: 25656, dtype: object
----------SIMILAR POSITION CUSTOMERS------------
                       video_name  frame_id        xmin        ymin  \
294886  20220314_